In [ ]:
# ! pip install kaggle

In [ ]:
# ! mkdir ~/.kaggle

In [ ]:
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ! kaggle datasets download hsankesara/flickr-image-dataset

In [ ]:
# !unzip -qq swintransformer.zip
# !unzip -qq flickr-image-dataset.zip

In [3]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import string

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from tensorflow.keras.applications import efficientnet
from tensorflow.keras.layers import TextVectorization
# import tensorflow_datasets as tfds

# ds, ds_info = tfds.load('coco_captions', split='train', with_info=True)
# fig = tfds.show_examples(ds, ds_info)


seed = 111
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
import sys
sys.path.insert(1, '../input/swim-model')
from model import SwinTransformer

In [ ]:
# from swintransformer import SwinTransformer

In [5]:

# Path to the images
IMAGES_PATH = "../input/flickr-image-dataset/flickr30k_images/flickr30k_images"

# # Desired image dimensions
IMAGE_SIZE = (224, 224)

# Vocabulary size
VOCAB_SIZE = 9000

# Fixed length allowed for any sequence
SEQ_LENGTH = 20

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 64
EPOCHS = 52
AUTOTUNE = tf.data.AUTOTUNE

In [6]:
LOCAL_FLICKR_PATH = '../input/flickr-image-dataset/flickr30k_images/'
annotation_file = LOCAL_FLICKR_PATH + 'results.csv'
LOCAL_IMG_PATH = LOCAL_FLICKR_PATH + 'flickr30k_images/'

!ls {LOCAL_IMG_PATH} | wc

In [7]:
#Image Path
# Image path given ------ used glob to get list of all images
image_path = "../input/flickr-image-dataset/flickr30k_images/flickr30k_images/"
jpgs = os.listdir(image_path)
images = glob.glob(image_path+'*.jpg')



#Caption Path
# Read the captions with Pandas ---- changed coloumn names
caption_path = '../input/flickr-image-dataset/flickr30k_images/results.csv'
captions = pd.read_csv(caption_path, sep = '|')
df = pd.DataFrame(captions)
df = df.rename(columns={'image_name' : 'filename',' comment_number': 'index', ' comment': 'caption'}, inplace = False)
df = df.reindex(columns =['index','filename','caption'])
data = df[df['caption'].notnull()]



#Printing DataFrame
# DataFrame name is Data
print("Total Images in Dataset = {}".format(len(jpgs)))
data.head()

In [8]:
# Made a list of images, above list had sets of images, this also has replciated images 
# ( 5 same image paths of 5 captions)
all_img_name_vector = []
for annot in data["filename"]:
   full_image_path = image_path + annot
   all_img_name_vector.append(full_image_path)

#To know lenght of list
len(all_img_name_vector)

In [9]:
plt.imshow(plt.imread('../input/flickr-image-dataset/flickr30k_images/flickr30k_images/1000092795.jpg'))

In [10]:
data_with_seq = data.copy()
data_with_seq.caption = data_with_seq.caption.apply(lambda x: "<start> " + x.strip() + " <end>")

In [11]:
train_dict = {}
for i in range(len(all_img_name_vector[:-10000])):
  if train_dict.get(all_img_name_vector[i],None) == None:
    train_dict[all_img_name_vector[i]] = [data.caption.iloc[i]]
  else:
    train_dict[all_img_name_vector[i]].append(data.caption.iloc[i])

In [12]:
train_data = train_dict.copy()
for k, v in train_dict.items():
  if len(v) < 5:
    train_data.pop(k)

In [13]:
test_dict = {}
for i in range(len(all_img_name_vector[-10000:])):
  if test_dict.get(all_img_name_vector[i],None) == None:
    test_dict[all_img_name_vector[i]] = [data.caption.iloc[i]]
  else:
    test_dict[all_img_name_vector[i]].append(data.caption.iloc[i])

In [14]:
test_data = test_dict.copy()
for k, v in test_dict.items():
  if len(v) < 5:
    test_data.pop(k)

## Vectorizing the text data

We'll use the `TextVectorization` layer to vectorize the text data,
that is to say, to turn the
original strings into integer sequences where each integer represents the index of
a word in a vocabulary. We will use a custom string standardization scheme
(strip punctuation characters except `<` and `>`) and the default
splitting scheme (split on whitespace).

In [15]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
)
vectorization.adapt(data.caption.values)

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.3),
    ]
)


## Building a `tf.data.Dataset` pipeline for training

We will generate pairs of images and corresponding captions using a `tf.data.Dataset` object.
The pipeline consists of two steps:

1. Read the image from the disk
2. Tokenize all the five captions corresponding to the image

In [16]:

def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, captions):
    return decode_and_resize(img_path), vectorization(captions)


def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(len(images))
    dataset = dataset.map(process_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return dataset


# Pass the list of images and the list of corresponding captions
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))

valid_dataset = make_dataset(list(test_data.keys()), list(test_data.values()))


## Building the model

Our image captioning architecture consists of three models:

1. A CNN: used to extract the image features
2. A TransformerEncoder: The extracted image features are then passed to a Transformer
                    based encoder that generates a new representation of the inputs
3. A TransformerDecoder: This model takes the encoder output and the text data
                    (sequences) as inputs and tries to learn to generate the caption.

In [18]:

def get_cnn_model():
    base_model = keras.models.Sequential([
                                          keras.layers.Lambda(
                                              lambda data: keras.applications.imagenet_utils.preprocess_input(
                                                  tf.cast(data, tf.float32), mode="torch"),
                                                   input_shape=[*IMAGE_SIZE, 3]),
                                          SwinTransformer('swin_base_224', include_top=False, pretrained=True)])
    # efficientnet.EfficientNetB3(
    #     input_shape=(*IMAGE_SIZE, 3), include_top=False, weights="imagenet",
    # )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.0
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(embed_dim, activation="relu")

    def call(self, inputs, training, mask=None):
        inputs = self.layernorm_1(inputs)
        inputs = self.dense_1(inputs)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        out_1 = self.layernorm_2(inputs + attention_output_1)
        return out_1


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens = embedded_tokens * self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.embedding = PositionalEmbedding(
            embed_dim=EMBED_DIM, sequence_length=SEQ_LENGTH, vocab_size=VOCAB_SIZE
        )
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

        self.dropout_1 = layers.Dropout(0.3)
        self.dropout_2 = layers.Dropout(0.5)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


class ImageCaptioningModel(keras.Model):
    def __init__(
        self, cnn_model, encoder, decoder, num_captions_per_image=5, image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]


cnn_model = get_cnn_model()
encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)

# Model Training

In [30]:
# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

# EarlyStopping criteria
# early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)


# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True),
    
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
# Fit the model
caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=my_callbacks,
)

In [35]:
train_loss=[26.9275,18.8002,17.2580,16.4341,15.8876,15.4700,15.1338,14.8521,14.6026,14.3928,14.1907,14.0111,13.7009,13.5562,13.4302,13.3108,13.0832,12.9825,12.8888,12.8033,12.7566,12.6366]
val_loss=[19.1178,16.4775,15.2753,14.5390,13.9776,13.5779,13.2489,12.9007,12.6872,12.4396,12.2091,11.9688,11.8315,11.6511,11.4848,11.3461,11.2180,11.0490,10.9646,10.7944,10.7315,10.6324]
vocab="10000"
epoch='20'
time_of_epoch="8 min"
emb="512"
model="swin transformer"

In [36]:
import matplotlib.pyplot as plt
x=[i for i in range(len(val_loss))]
plt.figure(figsize=(6,6))
plt.plot(x,train_loss,label="train_loss")
plt.plot(x,val_loss,label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("model=swin transformer     vocab=10000        batch_size = 64      n_epochs = 22     time_of_epoch=8 min    emb=512 ")
plt.legend()
plt.show()

In [37]:
train_loss=[17.7250,16.9968,16.3065,15.7969,15.4110,15.0838,14.8068,14.5705,14.3611,14.1762,13.9982,13.8393,13.6898,13.5541,13.4355,13.3116,13.1957,13.0879,12.9939,12.8931,12.8036,12.7163,12.6301,12.5436,12.4753,12.4060,12.3328,12.2716,12.2022,12.1536,12.0788,12.0348,12.0348,11.9667,11.9150,11.8722,11.8157,11.7658,11.7224,11.6730,11.6257,11.5912,11.5414,11.5101,11.4517,11.4205,11.3940,11.3683,11.3214,11.2865,11.2571,11.2252]
val_loss  =[15.9586,15.0190,14.3488,13.8624,13.5438,13.1446,12.8397,12.5733,12.3711,12.1182,11.9441,11.7475,11.6333,11.4661,11.3414,11.2005,11.0793,10.9239,10.7995,10.7441,10.6363,10.5754,10.4139,10.3335,10.2568,10.1475,10.1002,10.0447,9.8908,9.8353,9.7905,9.7112,9.7112,9.7182,9.6517,9.5367,9.5034,9.4167,9.4034,9.3124,9.2971,9.2589,9.1639,9.1320,9.0487,9.0450,9.0282,8.9623,8.9436,8.9047,8.8269,8.8697]

In [38]:
import matplotlib.pyplot as plt
x=[i for i in range(len(val_loss))]
plt.figure(figsize=(6,6))
plt.plot(x,train_loss,label="train_loss")
plt.plot(x,val_loss,label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("model=swin transformer     vocab=9000        batch_size = 64      n_epochs = 52     time_of_epoch=8 min    emb=512 ")
plt.legend()
plt.show()

In [31]:
caption_model.save_weights('weights.h5')

In [ ]:
caption_model=keras.models.Sequential()
caption_model.load_model("./weights.h5")


In [ ]:
# caption_model.load_weights("./swin_weights_30k_13.h5")
reconstructed_model = keras.models.load_model("./weights.h5")

In [ ]:
!cat weights.h5

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="../input/swim-model/weights.h5", verbose=1)


In [ ]:
# Fit the model
caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)

In [44]:
vocab = vectorization.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
valid_images = list(test_data.keys())


def generate_caption():
    # Select a random image from the validation dataset
    sample_img = np.random.choice(valid_images)

    # Read the image from the disk
    sample_img = decode_and_resize(sample_img)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    plt.imshow(img)
    plt.show()

    # Pass the image to the CNN
    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    # Pass the image features to the Transformer encoder
    encoded_img = caption_model.encoder(img, training=False)

    # Generate the caption using the Transformer decoder
    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        tokenized_caption = vectorization([decoded_caption])[:, :-1]
        mask = tf.math.not_equal(tokenized_caption, 0)
        predictions = caption_model.decoder(
            tokenized_caption, encoded_img, training=False, mask=mask
        )
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = index_lookup[sampled_token_index]
        if sampled_token == " <end>":
            break
        decoded_caption += " " + sampled_token

    decoded_caption = decoded_caption.replace("<start> ", "")
    decoded_caption = decoded_caption.replace(" <end>", "").strip()
    print("Predicted Caption: ", decoded_caption)


# Check predictions for a few samples
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()
generate_caption()

In [45]:
vocab = vectorization.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
valid_images = list(test_data.keys())


def generate_caption(test_img):
    # Select a random image from the validation dataset
    # sample_img = np.random.choice(valid_images)

    # Read the image from the disk
    sample_img = decode_and_resize(test_img)
    # img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    # plt.imshow(img)
    # plt.show()

    # Pass the image to the CNN
    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    # Pass the image features to the Transformer encoder
    encoded_img = caption_model.encoder(img, training=False)

    # Generate the caption using the Transformer decoder
    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        tokenized_caption = vectorization([decoded_caption])[:, :-1]
        mask = tf.math.not_equal(tokenized_caption, 0)
        predictions = caption_model.decoder(
            tokenized_caption, encoded_img, training=False, mask=mask
        )
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = index_lookup[sampled_token_index]
        if sampled_token == " <end>":
            break
        decoded_caption += " " + sampled_token

    decoded_caption = decoded_caption.replace("<start> ", "")
    decoded_caption = decoded_caption.replace(" <end>", "").strip()
    # print("Predicted Caption: ", decoded_caption)
    return decoded_caption


# Check predictions for a few samples
# generate_caption()
# generate_caption()
# generate_caption()

In [41]:
from nltk.translate.bleu_score import corpus_bleu

def bleu():
  true_imgs = list(test_data.keys())[:100]
  actual = []
  for refs in list(test_data.values())[:100]:
    actual.append([x.split() for x in refs])
  preds = []
  i = 0
  for im in true_imgs:
    preds.append(generate_caption(im).split())
    print(i)
    i+=1

  print('BLEU-1: %f' % corpus_bleu(actual, preds, weights=(1.0, 0, 0, 0)))
  print('BLEU-2: %f' % corpus_bleu(actual, preds, weights=(0.5, 0.5, 0, 0)))
  print('BLEU-3: %f' % corpus_bleu(actual, preds, weights=(0.3, 0.3, 0.3, 0)))
  print('BLEU-4: %f' % corpus_bleu(actual, preds, weights=(0.25, 0.25, 0.25, 0.25)))
bleu()  

In [42]:
def test_cap(img):
    sample_img = decode_and_resize(img)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    plt.imshow(img)
    plt.show()

    # Pass the image to the CNN
    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    # Pass the image features to the Transformer encoder
    encoded_img = caption_model.encoder(img, training=False)

    # Generate the caption using the Transformer decoder
    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        tokenized_caption = vectorization([decoded_caption])[:, :-1]
        mask = tf.math.not_equal(tokenized_caption, 0)
        predictions = caption_model.decoder(
            tokenized_caption, encoded_img, training=False, mask=mask
        )
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = index_lookup[sampled_token_index]
        if sampled_token == " <end>":
            break
        decoded_caption += " " + sampled_token

    decoded_caption = decoded_caption.replace("<start> ", "")
    decoded_caption = decoded_caption.replace(" <end>", "").strip()
    print("Predicted Caption: ", decoded_caption)

In [48]:
test_cap('../input/test-images/test images/landscape.jpg')